In [1]:
from pycalphad import Database, Model
from pycalphad.core.utils import NumPyPrinter
from pycalphad.core.autograd_utils import build_functions as build_functions_ag
from pycalphad.core.sympydiff_utils import make_gradient_from_graph
import pycalphad.variables as v
from sympy import lambdify
import numpy as np
import algopy
from algopy import UTPM
import itertools
mod = Model(Database('2016-04-01-AlNi.tdb'), ['AL', 'NI', 'VA'], 'FCC_L12')
print(tuple([v.T] + mod.site_fractions))

inp_arr = np.tile([[300,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1], [600, 0.4, 0.6,0.4,0.6,0.4,0.6,0.4,0.6,1]], (300,1))
#inp_arr = np.array([[600, 0.4, 0.6,0.4,0.6,0.4,0.6,0.4,0.6,1]])
inp_arr = np.random.RandomState(1769).rand(10000, 10)
inp_arr[0, 0] = 1600
inp_arr[1, 0] = 1600
inp_arr[:, -1] = 1 - inp_arr[:, 1]
inp_arr = np.expand_dims(inp_arr[1], axis=0)
#inp_arr = np.array([[300, 1e-4, 1-1e-4]])
print(inp_arr)

(T, FCC_L120AL, FCC_L120NI, FCC_L121AL, FCC_L121NI, FCC_L122AL, FCC_L122NI, FCC_L123AL, FCC_L123NI, FCC_L124VA)
[[  1.60000000e+03   5.40478524e-01   5.16085552e-01   4.98416268e-01
    6.99731480e-01   5.92997785e-01   2.29976414e-01   9.22020353e-01
    4.08862360e-01   4.59521476e-01]]


In [2]:
obj_ag, grad_ag, hess_ag = build_functions_ag(mod.ast, mod.variables)
from numba.compiler import Pipeline, native_lowering_stage
from numba.lowering import Lower
%time grad_numba, hess_numba = make_gradient_from_graph(mod.ast, mod.variables)

CPU times: user 34.9 s, sys: 90 ms, total: 35 s
Wall time: 35 s


In [3]:
import itertools
%time g3 = grad_numba(*[inp_arr[..., i] for i in range(inp_arr.shape[-1])])
%time g1 = grad_ag(*inp_arr.T)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 188 µs
CPU times: user 85 ms, sys: 1 ms, total: 86 ms
Wall time: 85.6 ms


In [4]:
#o1 = obj_ag(*inp_arr.T)
%time h3 = hess_numba(*[inp_arr[..., i] for i in range(inp_arr.shape[-1])])
%time h1 = hess_ag(*inp_arr.T)

CPU times: user 1e+03 µs, sys: 0 ns, total: 1e+03 µs
Wall time: 235 µs
CPU times: user 2.29 s, sys: 7 ms, total: 2.29 s
Wall time: 2.27 s


In [5]:
g1

array([[ -3.20865747e+01,  -4.67034156e+04,  -5.65974961e+04,
         -4.43548465e+04,  -4.55839415e+04,  -6.11081893e+04,
         -8.16752838e+04,  -3.79620528e+04,  -4.48707719e+04,
         -1.42145241e+05]])

In [6]:
g3

array([[ -3.20865747e+01,  -4.67034156e+04,  -5.65974961e+04,
         -4.43548465e+04,  -4.55839415e+04,  -6.11081893e+04,
         -8.16752838e+04,  -3.79620528e+04,  -4.48707719e+04,
         -1.42145241e+05]])

In [7]:
import numpy.testing

numpy.testing.assert_allclose(g1, g3)
print('equivalent')
numpy.testing.assert_allclose(h1, h3)
print('equivalent')

equivalent


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(mismatch 5.0%)
 x: array([[[ -7.699249e-03,  -2.605680e+01,  -2.771574e+01,  -2.395319e+01,
          -2.308483e+01,  -3.499028e+01,  -3.442843e+01,  -2.165418e+01,
          -1.892117e+01,  -8.372728e+01],...
 y: array([[[  7.700152e-03,  -2.605680e+01,  -2.771574e+01,  -2.395319e+01,
          -2.308483e+01,  -3.499028e+01,  -3.442843e+01,  -2.165418e+01,
          -1.892117e+01,  -8.372728e+01],...

In [ ]:
h1

In [ ]:
h3

In [ ]:
h1-h3